In [3]:
from sklearn.datasets import fetch_openml
import pandas as pd
import numpy as np

In [4]:
mnist = fetch_openml('mnist_784', version=1)

print(mnist['data'].iloc[0, :])

/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.8/site-packages/sklearn/datasets/_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


pixel1      0.0
pixel2      0.0
pixel3      0.0
pixel4      0.0
pixel5      0.0
           ... 
pixel780    0.0
pixel781    0.0
pixel782    0.0
pixel783    0.0
pixel784    0.0
Name: 0, Length: 784, dtype: float64


In [5]:
X = mnist['data'].astype(np.uint8)

In [6]:
print(X)

       pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  \
0           0       0       0       0       0       0       0       0       0   
1           0       0       0       0       0       0       0       0       0   
2           0       0       0       0       0       0       0       0       0   
3           0       0       0       0       0       0       0       0       0   
4           0       0       0       0       0       0       0       0       0   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
69995       0       0       0       0       0       0       0       0       0   
69996       0       0       0       0       0       0       0       0       0   
69997       0       0       0       0       0       0       0       0       0   
69998       0       0       0       0       0       0       0       0       0   
69999       0       0       0       0       0       0       0       0       0   

       pixel10  ...  pixel7

In [7]:
y = mnist['target'].sort_values().astype(np.uint8)

In [8]:
X = X.reindex(y.index)

In [9]:
X_train, X_test = X[:56000], X[56000:]
y_train, y_test = y[:56000], y[56000:]

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(56000, 784) (14000, 784)
(56000,) (14000,)


In [10]:
print(pd.unique(y_train))
print(pd.unique(y_test))

[0 1 2 3 4 5 6 7]
[7 8 9]


In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
print(pd.unique(y_train))
print(pd.unique(y_test))

[6 9 1 4 0 2 8 3 5 7]
[6 5 4 1 3 0 8 2 9 7]


In [14]:
from sklearn.linear_model import SGDClassifier

In [15]:
sgd_clf = SGDClassifier(random_state=42)

In [16]:
y_train_0 = (y_train == 0)
y_test_0 = (y_test == 0)

In [17]:
sgd_clf.fit(X_train, y_train_0)

SGDClassifier(random_state=42)

In [18]:
print(sgd_clf.predict([mnist["data"].iloc[0, :], mnist["data"].iloc[1, :]]))

[False  True]


/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(


In [19]:
y_train_pred = sgd_clf.predict(X_train)
y_test_pred = sgd_clf.predict(X_test)

acc_train = sum(y_train_pred == y_train_0)/len(y_train_0)
acc_test = sum(y_test_pred == y_test_0)/len(y_test_0)

In [20]:
print(acc_train, acc_test)

0.9832142857142857 0.9818571428571429


In [21]:
acc = [acc_train, acc_test]

In [22]:
import pickle

with open('sgd_acc.pkl', 'wb') as f:
    pickle.dump(acc, f)

In [23]:
from sklearn.model_selection import cross_val_score

In [25]:
score = cross_val_score(sgd_clf, X_train, y_train_0, cv=3, scoring='accuracy', n_jobs=-1)

<class 'numpy.ndarray'>


In [27]:
with open('sgd_cva.pkl', 'wb') as f:
    pickle.dump(score, f)

In [32]:
sgd_m_clf = SGDClassifier(random_state=42, n_jobs=-1)

In [33]:
sgd_m_clf.fit(X_train, y_train)

SGDClassifier(n_jobs=-1, random_state=42)

In [36]:
sgd_m_clf.predict([mnist['data'].iloc[0, :]])

/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(


array([5], dtype=uint8)

In [41]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict

In [42]:
y_test_pred = cross_val_predict(sgd_m_clf, X_test, y_test, cv=3, n_jobs=-1)
conf_mx = confusion_matrix(y_test, y_test_pred)

In [45]:
with open('sgd_cmx.pkl', 'wb') as f:
    pickle.dump(conf_mx, f)